· Coronavirus tweets NLP - Text Classification


Veriler üzerinde Metin Sınıflandırması gerçekleştirin. Tweetler Twitter'dan çekildi ve ardından manuel etiketleme yapıldı.
Herhangi bir gizlilik endişesinden kaçınmak için adlara ve kullanıcı adlarına kodlar verilmiştir.

Sütunlar:
1) Konum
2) Tweet at
3) Orijinal Tweet
4) Etiket

https://www.kaggle.com/datasets/datatattle/covid-19-nlp-text-classification

In [1]:
#NLTK - Natural Language Toolkit

In [2]:
!pip install nltk

In [3]:
import pandas as pd
import numpy as np
import scipy as sp
import nltk

import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob

import sys 
from nltk.stem import SnowballStemmer
from langdetect import detect
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from wordcloud import WordCloud, STOPWORDS
import os
from PIL import Image
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

''''# NLP Projesi icin yapmamiz gerekenler
1-Herseyi kucuk harfe cevir
2-Noktalama isaretlerini kaldir
3-Rakamlari kaldir
4-satir sonlari \n kaldir
5- Gereksiz kelimeleri cikar Stopword
6-Tokenize islemi
7-Lemma ve Stemma
8 - Vectorizer rakama ceviriyor'''

In [4]:
df1= pd.read_csv("Corona_NLP_train.csv" ,encoding='latin-1' , sep=',' )
df2 = pd.read_csv("Corona_NLP_test.csv" , encoding='latin-1' ,sep=',')

In [5]:
df1.drop(["UserName" , "ScreenName" , "Location"] , axis = 1 , inplace = True)
df2.drop(["UserName" , "ScreenName" , "Location"], axis = 1 , inplace = True)
#kişi ismi, kullanıcı adı, yer bilgilerini veri setinden sildik

In [6]:
print(df1.info())
print(df2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41157 entries, 0 to 41156
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   TweetAt        41157 non-null  object
 1   OriginalTweet  41157 non-null  object
 2   Sentiment      41157 non-null  object
dtypes: object(3)
memory usage: 964.7+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3798 entries, 0 to 3797
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   TweetAt        3798 non-null   object
 1   OriginalTweet  3798 non-null   object
 2   Sentiment      3798 non-null   object
dtypes: object(3)
memory usage: 89.1+ KB
None


In [7]:
df1["Sentiment"].value_counts()

Positive              11422
Negative               9917
Neutral                7713
Extremely Positive     6624
Extremely Negative     5481
Name: Sentiment, dtype: int64

In [8]:
df2["Sentiment"].value_counts()

Negative              1041
Positive               947
Neutral                619
Extremely Positive     599
Extremely Negative     592
Name: Sentiment, dtype: int64

In [9]:
pip install category_encoders

Note: you may need to restart the kernel to use updated packages.


In [10]:
from category_encoders import OrdinalEncoder

In [11]:
maplist = [{'col': 'Sentiment', 'mapping': {'Extremely Negative': 0, 'Negative': 1,'Neutral': 2, 'Positive': 3, 'Extremely Positive': 4}}]
enc = OrdinalEncoder(mapping=maplist)
df1 = enc.fit_transform(df1)
df2 = enc.transform(df2) #sentiment analizi sonuçlarına sayısal değer atadık


In [12]:
ytrain = df1["Sentiment"]
ytest = df2["Sentiment"]

xtrain = df1.drop("Sentiment" , axis = 1)
xtest = df2.drop("Sentiment" , axis = 1)

In [13]:
ytrain

0        2
1        3
2        3
3        3
4        0
        ..
41152    2
41153    0
41154    3
41155    2
41156    1
Name: Sentiment, Length: 41157, dtype: int64

In [14]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
 # 'Türler' sütunundaki etiketleri kodladık

xtrain['TweetAt']= label_encoder.fit_transform(xtrain['TweetAt'])
xtest['TweetAt']= label_encoder.fit_transform(xtest['TweetAt'])

# NLP Veri Ön Hazırlıkları

In [15]:
df1['OriginalTweet']=df1['OriginalTweet'].str.lower() #Butun sutunu kucuk harfe cevirir
df1['OriginalTweet']=df1['OriginalTweet'].str.replace('[^\w\s]','') # Noktalama isaretlerini kaldirir
df1['OriginalTweett']=df1['OriginalTweet'].str.replace('\d+','') # Rakamlari kaldirir
df1['OriginalTweet']=df1['OriginalTweet'].str.replace('\n','') # yeni satirlari kaldir
df1['OriginalTweet']=df1['OriginalTweet'].str.replace('\r','') #Enter i kaldir

/var/folders/3x/n1kkdkgn5m9c3thf3ctlb_xm0000gn/T/ipykernel_5009/371967069.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df1['OriginalTweet']=df1['OriginalTweet'].str.replace('[^\w\s]','') # Noktalama isaretlerini kaldirir
/var/folders/3x/n1kkdkgn5m9c3thf3ctlb_xm0000gn/T/ipykernel_5009/371967069.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df1['OriginalTweett']=df1['OriginalTweet'].str.replace('\d+','') # Rakamlari kaldirir


In [16]:
df2['OriginalTweet']=df2['OriginalTweet'].str.lower() #Butun sutunu kucuk harfe cevirir
df2['OriginalTweet']=df2['OriginalTweet'].str.replace('[^\w\s]','') # Noktalama isaretlerini kaldirir
df2['OriginalTweett']=df2['OriginalTweet'].str.replace('\d+','') # Rakamlari kaldirir
df2['OriginalTweet']=df2['OriginalTweet'].str.replace('\n','') # yeni satirlari kaldir
df2['OriginalTweet']=df2['OriginalTweet'].str.replace('\r','') #Enter i kaldir

/var/folders/3x/n1kkdkgn5m9c3thf3ctlb_xm0000gn/T/ipykernel_5009/3888126497.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df2['OriginalTweet']=df2['OriginalTweet'].str.replace('[^\w\s]','') # Noktalama isaretlerini kaldirir
/var/folders/3x/n1kkdkgn5m9c3thf3ctlb_xm0000gn/T/ipykernel_5009/3888126497.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df2['OriginalTweett']=df2['OriginalTweet'].str.replace('\d+','') # Rakamlari kaldirir


In [17]:
df1.head()

,TweetAt,OriginalTweet,Sentiment,OriginalTweett
0,16-03-2020,menyrbie phil_gahan chrisitv httpstcoifz9fan2p...,2,menyrbie phil_gahan chrisitv httpstcoifzfanpa ...
1,16-03-2020,advice talk to your neighbours family to excha...,3,advice talk to your neighbours family to excha...
2,16-03-2020,coronavirus australia woolworths to give elder...,3,coronavirus australia woolworths to give elder...
3,16-03-2020,my food stock is not the only one which is emp...,3,my food stock is not the only one which is emp...
4,16-03-2020,me ready to go at supermarket during the covid...,0,me ready to go at supermarket during the covid...


In [18]:
df1

,TweetAt,OriginalTweet,Sentiment,OriginalTweett
0,16-03-2020,menyrbie phil_gahan chrisitv httpstcoifz9fan2p...,2,menyrbie phil_gahan chrisitv httpstcoifzfanpa ...
1,16-03-2020,advice talk to your neighbours family to excha...,3,advice talk to your neighbours family to excha...
2,16-03-2020,coronavirus australia woolworths to give elder...,3,coronavirus australia woolworths to give elder...
3,16-03-2020,my food stock is not the only one which is emp...,3,my food stock is not the only one which is emp...
4,16-03-2020,me ready to go at supermarket during the covid...,0,me ready to go at supermarket during the covid...
...,...,...,...,...
41152,14-04-2020,airline pilots offering to stock supermarket s...,2,airline pilots offering to stock supermarket s...
41153,14-04-2020,response to complaint not provided citing covi...,0,response to complaint not provided citing covi...
41154,14-04-2020,you know itâs getting tough when kameronwilds ...,3,you know itâs getting tough when kameronwilds ...
41155,14-04-2020,is it wrong that the smell of hand sanitizer i...,2,is it wrong that the smell of hand sanitizer i...


In [19]:
df1.shape

(41157, 4)

In [20]:
df2

,TweetAt,OriginalTweet,Sentiment,OriginalTweett
0,02-03-2020,trending new yorkers encounter empty supermark...,0,trending new yorkers encounter empty supermark...
1,02-03-2020,when i couldnt find hand sanitizer at fred mey...,3,when i couldnt find hand sanitizer at fred mey...
2,02-03-2020,find out how you can protect yourself and love...,4,find out how you can protect yourself and love...
3,02-03-2020,panic buying hits newyork city as anxious shop...,1,panic buying hits newyork city as anxious shop...
4,03-03-2020,toiletpaper dunnypaper coronavirus coronavirus...,2,toiletpaper dunnypaper coronavirus coronavirus...
...,...,...,...,...
3793,16-03-2020,meanwhile in a supermarket in israel people d...,3,meanwhile in a supermarket in israel people d...
3794,16-03-2020,did you panic buy a lot of nonperishable items...,1,did you panic buy a lot of nonperishable items...
3795,16-03-2020,asst prof of economics cconces was on nbcphila...,2,asst prof of economics cconces was on nbcphila...
3796,16-03-2020,gov need to do somethings instead of biar je r...,0,gov need to do somethings instead of biar je r...


In [21]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [22]:
vectorizer = TfidfVectorizer(max_features=8000)
bow_train = vectorizer.fit_transform(xtrain["OriginalTweet"])
features = vectorizer.get_feature_names_out()
xtrain_bow=pd.DataFrame(bow_train.toarray(), columns = features)
xtrain_bow

,00,000,00am,00pm,01,03,04,06,07,08,...,yyc,yâ,zambia,zealand,zero,zimbabwe,zombie,zombies,zone,zoom
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41152,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
41153,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
41154,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
41155,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
xtrain_bow["zone"].value_counts()

0.000000    41134
0.311651        1
0.264563        1
0.258502        1
0.257087        1
0.309607        1
0.246882        1
0.234649        1
0.222425        1
0.227430        1
0.225887        1
0.300836        1
0.270933        1
0.261711        1
0.234287        1
0.226250        1
0.280613        1
0.341972        1
0.335291        1
0.358021        1
0.290574        1
0.285685        1
0.225633        1
0.384936        1
Name: zone, dtype: int64

In [24]:
bow_test = vectorizer.transform(xtest["OriginalTweet"])
features = vectorizer.get_feature_names_out()
xtest_bow=pd.DataFrame(bow_test.toarray(), columns = features)

In [25]:
xtrain_bow  = pd.concat([xtrain["TweetAt"], xtrain_bow] , axis =1)
xtest_bow  = pd.concat([xtest["TweetAt"], xtest_bow] , axis =1)

In [26]:
df2.shape

(3798, 4)

In [ ]:
star=df[(df.stars==1)|(df.stars==5)]

In [ ]:
star.reset_index(drop=True,inplace=True)

In [30]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
from sklearn.svm import SVC
svm = SVC() #makine öğrenmesi algoritmalarını çağırdık

In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [32]:
l=LogisticRegression()

In [33]:
from sklearn.metrics import accuracy_score

In [34]:
l.fit(xtrain_bow,ytrain)

/Users/bmaho/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [35]:
pred=l.predict(xtest_bow)

In [36]:
accuracy_score(pred,ytest)

0.41311216429699843

In [39]:
from sklearn.naive_bayes import MultinomialNB

In [40]:
m=MultinomialNB()

In [41]:
pred2=m.fit(xtrain_bow,ytrain).predict(xtest_bow)

In [42]:
accuracy_score(pred2,ytest)

0.42706687730384413

In [ ]:
# MultinomialNB de 0.42 çıktı
# Lojistik Regresyonda 0.41 çıktı
